In [33]:
from deep_sort_realtime.deepsort_tracker import DeepSort
from insightface.app import FaceAnalysis
import insightface
import cv2
from funcs import convert_detections_to_bbs_format
app = FaceAnalysis(providers=['CUDAExecutionProvider'], allowed_modules=['detection'])
app.prepare(ctx_id=0, det_size=(640, 640))
tracker = DeepSort(max_age=30)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/stargroup/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/stargroup/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/stargroup/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/stargroup/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/stargroup/.insightface/models/buffalo_l/w600k_r50.onnx recognition
set det-size: (640, 640)


In [ ]:
tracker = DeepSort(max_age=5, )
img = cv2.imread('/home/stargroup/new/camyolo/bus.jpg')
datection = app.get(img)
bbs = convert_detections_to_bbs_format(datection)
tracks = tracker.update_tracks(bbs, frame=img)
print(type(tracker))

In [ ]:
for track in tracks:
    print(int(track.track_id[0]))
    if not track.is_confirmed() or track.time_since_update > 1:
        continue
    track_id = track.track_id
    print(int(track_id))
    ltrb = track.to_ltrb()
    print(int(ltrb))

In [37]:
cap = cv2.VideoCapture('/home/stargroup/new/saved_videos/output_video_2024-03-11_13-15-13.avi')

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    datection = app.get(frame)
    bbs = convert_detections_to_bbs_format(datection)
    tracks = tracker.update_tracks(bbs, frame=frame)
    for track in tracks:
        print(track)
        if not track.is_confirmed() or track.time_since_update > 1:
            # Пропускаем не подтвержденные треки или треки, которые не обновлялись
            continue
        
        track_id = track.track_id  # ID трека
        ltrb = track.to_ltrb()
    
        # Отрисовка рамки вокруг объекта и ID трека на изображении
        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0, 255, 0), 3)
        cv2.putText(frame, str(track_id), (int(ltrb[0]), int(ltrb[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        cv2.imshow("Tracks", frame)
        if cv2.waitKey(2) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()


error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
